In [ ]:
# Install any dependencies
!pip install pandas
!pip install praw
!pip install python-dotenv
!pip install pyarrow

In [ ]:
# Set the head number to the amount of entries you want to load in minus one
ENTRIES_COUNT = 1001

In [ ]:
import pandas as pd
import praw
import os
from os.path import join, dirname
from dotenv import main

# Make sure you create a .env file and fill in all the necessary information in the same folder as this script!
main.load_dotenv(join(dirname(os.path.realpath('__file__')), '.env'))

reddit = praw.Reddit(
   client_id=os.environ.get("CLIENT_ID"),
   client_secret=os.environ.get("CLIENT_SECRET"),
   user_agent="CMV_Scraper",
)


In [ ]:
# load the data
import tarfile
import os.path
import json
import re
from bz2 import BZ2File
from urllib import request
from io import BytesIO

import numpy as np


fname = "cmv.tar.bz2"
url = "https://chenhaot.com/data/cmv/" + fname

# download if not exists
if not os.path.isfile(fname):
    f = BytesIO()
    with request.urlopen(url) as resp, open(fname, 'wb') as f_disk:
        data = resp.read()
        f_disk.write(data)  # save to disk too
        f.write(data)
        f.seek(0)
else:
    f = open(fname, 'rb')




In [ ]:
#tar = tarfile.open(fileobj=f, mode="r:bz2")
tar = tarfile.open(fileobj=f, mode="r")

# Extract the file we are interested in

train_fname = "op_task/train_op_data.jsonlist.bz2"
test_fname = "op_task/heldout_op_data.jsonlist.bz2"

train_bzlist = tar.extractfile(train_fname)

In [ ]:
# Deserialize the JSON list
original_posts_train = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(train_bzlist)
]

In [ ]:
original_posts_train

In [ ]:
# Load the jsonlist file into a dataframe
#df = pd.read_json(original_posts_train, orient='list', lines=True)
df = pd.DataFrame(original_posts_train)

In [ ]:
# Function to check if the posts still exists on reddit
def try_get_post(post_id):
    try:
        submission = reddit.submission(id=post_id)
        submission.name
        return True
    except Exception as e:
        return False

In [ ]:
# Removes > sign and the template message at the end of a message
def cleanup(cmv_post):
    lines = [line for line in cmv_post.splitlines()
            if not line.lstrip().startswith("&gt;")
            and not line.lstrip().startswith("____")
            and not line.lstrip().startswith("So go forth and CMV, noble redditors!")
            and "edit" not in " ".join(line.lower().split()[:2])
            ]
    #print(lines)
    return "\n".join(lines)

# Create the function that will be handling all the data gathering
def get_top_comment_and_clean_data(post_id):
    #print(post_id.lstrip("t3_"))
    last_author = ""
    # Grab the post
    submission = reddit.submission(id=post_id.lstrip("t3_"))
    print(submission)

    # Grab the highest rated comment on root layer
    submission.submission_type = 'best'
    submission.comments.replace_more(limit=0)
    replies = list(submission.comments)[0].replies.list()

    # Just some variables
    responses = pros = cons = []

    # If the post author doesn't exist this submission was deleted (submission.deleted doesn't work)
    if type(submission.author) == type(None):
        last_author = "[deleted]"
    else:
        last_author = submission.author.name

    is_pro_argument = False

    for comment in replies:

        #print(comment)
        responses.append(comment.body)

        # Sometimes for some reason duplicate entries exist
        # Also remove automated message with "Δ" in it
        if comment.body in responses or "Δ" in responses:
            continue

        comment.body = comment.body.replace("[deleted]","")

        # If redditor object doesn't exist, the account is invalid/deleted
        if type(comment.author) == type(None):
            continue

        author = comment.author.name

        # Assume that whenever the user changes, they are countering the previous person
        if author != last_author:
            is_pro_argument = !is_pro_argument


        # Add to the respective argument type        
        if is_pro_argument:
            pros.append(comment.body)
        else:
            cons.append(comment.body)
        
        last_author = comment.author.name
        
    # Pros = arguments for the Title of this post
    # Cons = arguments against the title of this post
    
    return pros, cons, responses

In [ ]:
df.head(100)

In [ ]:

print(f"Loading in {ENTRIES_COUNT} posts")
dataset = df.head(ENTRIES_COUNT)


In [ ]:
# the name column does some weird sh** because dataframes already have a name property, so migrate to a different column name

import warnings
warnings.filterwarnings('ignore')

dataset["post_id"] = dataset["name"]
warnings.filterwarnings('default')

In [ ]:
%%time

# Reset variables for if we run this multiple times
all_pros = []
all_names = []
all_titles = []
all_sources = []

# load in our data. this will take a while.

for i in range(dataset.shape[0]):
    post = dataset.iloc[i]

    if type(post) == type(None):
        continue

    assert(post.post_id != i)

    pros, _cons, _response = get_top_comment_and_clean_data(post.post_id)

    # if type(post.name) == int:
    #     continue
    # if type(pros) == int:
    #     continue
    if post.title == "[deleted]":
        continue

    pros = " ".join(pros)
    pros = pros.replace("[deleted]","")

    modified_title = post.title.replace('CMV', "Change my mind")
    post.selftext = cleanup(post.selftext)
    all_titles.append(modified_title + " " + post.selftext)
    all_pros.append(pros)
    all_names.append(post.name)
    all_sources.append(f"https://reddit.com/r/changemyview/comments/{post.post_id}")
    #print(post.title)

    print(f"\n Loading entry {i}/{dataset.shape[0]}: {modified_title}")


In [ ]:
# Place it all into a Pandas Dataframe
clean_df = pd.DataFrame({
    "INSTRUCTION": all_titles,
    "RESPONSE": all_pros,
    "SOURCE": all_sources
}, index=all_names
)

In [ ]:
clean_df.head(9)

In [ ]:
# Create Apache Paquete file

import pyarrow as pa
import pyarrow.parquet as pq

table = pa.Table.from_pandas(clean_df)
pq.write_table(table,"output.parquet")

In [ ]:
# Test to see if it was sucessful
table = pq.read_table("output.parquet")
table.to_pandas()